## <center>MDS472C: NATURAL LANGUAGE PROCESSING</center>

## <center>LAB4: Revision and Word Sense Disambiguation (WSD) 
</center>

In [1]:
import re
from collections import defaultdict, Counter
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from nltk.metrics.distance import edit_distance
import math
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

In [2]:
# Download once if not already
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\Neelanjan
[nltk_data]     Dutta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Neelanjan Dutta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Neelanjan
[nltk_data]     Dutta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Neelanjan
[nltk_data]     Dutta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Neelanjan
[nltk_data]     Dutta\AppData\Roaming\nltk_data...


True

# Question 1: Consider the following two documents: 

**Doc 1: I am a student, and I currently take MDS472C. I was a student in MDS331 last trimester.** <br>
**Doc 2: I was a student. I have taken MDS472C.** <br><br>

**(a.) Create a dictionary with the positional index of the document.** <br>
**(b.)Find the positional indexes for the words “student” and “MDS472C”.** <br>
**(c.) Find the positional indexes for any given set of words as user input.**

In [3]:
# Documents
documents = {
    1: "I am a student, and I currently take MDS472C. I was a student in MDS331 last trimester.",
    2: "I was a student. I have taken MDS472C."
}

# Preprocessing and Tokenization
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9]+', ' ', text)
    return text.split()

# Build Positional Index
def build_positional_index(docs):
    index = defaultdict(lambda: defaultdict(list))
    for doc_id, text in docs.items():
        tokens = preprocess(text)
        print(f"\nDoc {doc_id} Tokens: {tokens}")
        for pos, token in enumerate(tokens):
            index[token][doc_id].append(pos)
    return index

# Display Index for Specific Word
def display_index_for(word, index):
    word = word.lower()
    if word in index:
        print(f"'{word}' found at: {dict(index[word])}")
    else:
        print(f"'{word}' not found in any document.")

# Full Positional Index
def display_full_index(index):
    print("\nFULL POSITIONAL INDEX:")
    for word, posting in sorted(index.items()):
        print(f"{word}: {dict(posting)}")

# Main Menu Loop
def main():
    print("Building Positional Index...")
    index = build_positional_index(documents)
    print("\nPositional Index Created!")

    while True:
        print("\n======== MENU ========")
        print("1. Show Full Positional Index")
        print("2. Find Positions for 'student'")
        print("3. Find Positions for 'MDS472C'")
        print("4. Search Word(s) of Your Choice")
        print("5. Exit")
        print("======================")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            display_full_index(index)
        elif choice == '2':
            display_index_for('student', index)
        elif choice == '3':
            display_index_for('MDS472C', index)
        elif choice == '4':
            words = input("Enter word(s) separated by spaces: ").split()
            for word in words:
                display_index_for(word, index)
        elif choice == '5':
            print("Exiting.")
            break
        else:
            print("Invalid choice. Try again.")

# Run the program
main()

Building Positional Index...

Doc 1 Tokens: ['i', 'am', 'a', 'student', 'and', 'i', 'currently', 'take', 'mds472c', 'i', 'was', 'a', 'student', 'in', 'mds331', 'last', 'trimester']

Doc 2 Tokens: ['i', 'was', 'a', 'student', 'i', 'have', 'taken', 'mds472c']

Positional Index Created!

======== MENU ========
1. Show Full Positional Index
2. Find Positions for 'student'
3. Find Positions for 'MDS472C'
4. Search Word(s) of Your Choice
5. Exit
Enter your choice (1-5): 1

FULL POSITIONAL INDEX:
a: {1: [2, 11], 2: [2]}
am: {1: [1]}
and: {1: [4]}
currently: {1: [6]}
have: {2: [5]}
i: {1: [0, 5, 9], 2: [0, 4]}
in: {1: [13]}
last: {1: [15]}
mds331: {1: [14]}
mds472c: {1: [8], 2: [7]}
student: {1: [3, 12], 2: [3]}
take: {1: [7]}
taken: {2: [6]}
trimester: {1: [16]}
was: {1: [10], 2: [1]}

======== MENU ========
1. Show Full Positional Index
2. Find Positions for 'student'
3. Find Positions for 'MDS472C'
4. Search Word(s) of Your Choice
5. Exit
Enter your choice (1-5): 2
'student' found at: {1: [

**Goal: Efficiently index and search words based on their exact positions in a set of documents. This is useful in information retrieval systems (like search engines) where we not only care whether a word exists, but also where it occurs.** <br>

**1. Documents to Index:** We start with two small example documents. Each document has text that we want to search through later.<br>

> - Doc 1: "I am a student, and I currently take MDS472C..."
> - Doc 2: "I was a student. I have taken MDS472C."

**2. Preprocessing the Text:** Each document is cleaned by: <br>
> - Converting to lowercase.
> - Removing all punctuation and special characters.
> - Splitting the cleaned sentence into a list of words (tokens).
> - Example: <br>
> Original → "I am a student."
> Preprocessed → ['i', 'am', 'a', 'student']

**3. Build the Positional Index:** We create a dictionary where: <br>
> - Each word maps to a dictionary of document IDs.
> - Each document ID maps to a list of positions (indices) where the word appears in that document.
> - Example:
> 'student': {1: [3, 10], 2: [3]} <br>
> This means: <br>
> In Doc 1, "student" appears at positions 3 and 10. <br>
> In Doc 2, it appears at position 3.

**4. User Interaction via Menu:** The program gives a simple menu where the user can: <br>
> - View the full positional index.
> - Search for a specific word (like "student" or "MDS472C").
> - Type in any word(s) to find their positions.
> - Exit the program.

**5. Output Example:** If the user searches for "student", the output will show: <br>
> 'student' found at: {1: [3, 10], 2: [3]}

# Question 2: Prepare a word matrix for the Doc 1 and Doc2

In [4]:
# Step 1: Input Documents
documents = {
    "Doc1": "I am a student, and I currently take MDS472C. I was a student in MDS331 last trimester.",
    "Doc2": "I was a student. I have taken MDS472C."
}

# Step 2: Preprocessing function
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9]+', ' ', text)  # remove punctuation
    return text.split()

# Step 3: Tokenize all documents
def tokenize_documents(docs):
    tokenized = {}
    for doc_name, content in docs.items():
        tokens = preprocess(content)
        tokenized[doc_name] = tokens
        print(f"\n{doc_name} Tokens: {tokens}")
    return tokenized

# Step 4: Build vocabulary
def build_vocabulary(tokenized_docs):
    vocab = sorted(set(word for tokens in tokenized_docs.values() for word in tokens))
    print(f"\nVocabulary (Unique Terms): {vocab}")
    return vocab

# Step 5: Create term-document matrix
def create_term_doc_matrix(vocab, tokenized_docs):
    matrix = defaultdict(dict)
    for term in vocab:
        for doc_name, tokens in tokenized_docs.items():
            matrix[term][doc_name] = tokens.count(term)  # frequency
    return matrix

# Step 6: Print the matrix nicely
def display_matrix(matrix, doc_names):
    print("\nTERM-DOCUMENT MATRIX (Counts):\n")
    header = ["Term"] + doc_names
    print("{:<15} {:<10} {:<10}".format(*header))
    print("-" * 35)
    for term, counts in matrix.items():
        row = [term] + [str(counts.get(doc, 0)) for doc in doc_names]
        print("{:<15} {:<10} {:<10}".format(*row))

# Step 7: Main menu
def main():
    print("Processing Documents and Creating Word Matrix...")
    tokenized_docs = tokenize_documents(documents)
    vocab = build_vocabulary(tokenized_docs)
    matrix = create_term_doc_matrix(vocab, tokenized_docs)
    print("\nWord Matrix Ready!")

    while True:
        print("\n======== MENU ========")
        print("1. Show Vocabulary")
        print("2. Show Word Matrix (Term-Document)")
        print("3. Search Term Frequency in Documents")
        print("4. Exit")
        print("======================")

        choice = input("Enter your choice (1-4): ")

        if choice == '1':
            print("\nVocabulary Terms:")
            for term in vocab:
                print(f"• {term}")
        elif choice == '2':
            display_matrix(matrix, list(documents.keys()))
        elif choice == '3':
            word = input("Enter the term to search: ").lower()
            if word in matrix:
                print(f"\nFrequencies of '{word}':")
                for doc in documents:
                    print(f"{doc}: {matrix[word].get(doc, 0)}")
            else:
                print(f"Term '{word}' not found in vocabulary.")
        elif choice == '4':
            print("Exiting.")
            break
        else:
            print("Invalid choice. Try again.")

# Run the program
main()

Processing Documents and Creating Word Matrix...

Doc1 Tokens: ['i', 'am', 'a', 'student', 'and', 'i', 'currently', 'take', 'mds472c', 'i', 'was', 'a', 'student', 'in', 'mds331', 'last', 'trimester']

Doc2 Tokens: ['i', 'was', 'a', 'student', 'i', 'have', 'taken', 'mds472c']

Vocabulary (Unique Terms): ['a', 'am', 'and', 'currently', 'have', 'i', 'in', 'last', 'mds331', 'mds472c', 'student', 'take', 'taken', 'trimester', 'was']

Word Matrix Ready!

======== MENU ========
1. Show Vocabulary
2. Show Word Matrix (Term-Document)
3. Search Term Frequency in Documents
4. Exit
Enter your choice (1-4): 1

Vocabulary Terms:
• a
• am
• and
• currently
• have
• i
• in
• last
• mds331
• mds472c
• student
• take
• taken
• trimester
• was

======== MENU ========
1. Show Vocabulary
2. Show Word Matrix (Term-Document)
3. Search Term Frequency in Documents
4. Exit
Enter your choice (1-4): 2

TERM-DOCUMENT MATRIX (Counts):

Term            Doc1       Doc2      
-----------------------------------
a     

**Goal: Create a term-document matrix, which shows how many times each word (term) appears in each document. This helps in comparing documents based on word usage, and is a basic building block for text mining and information retrieval systems.**

**1. Input Documents:** <br>
> - We start with two short documents (Doc1 and Doc2). These are simple English sentences that contain overlapping and unique words. Example: <br>
> Doc1: Talks about being a student and mentions "MDS472C" and "MDS331". <br>
> Doc2: Also mentions being a student and includes "MDS472C".

**2. Preprocessing and Tokenization:** Each document is cleaned by:<br>
> - Lowercasing all characters. <br>
> - Removing punctuation and special characters. <br>
> - Splitting the sentence into words (called tokens). <br>
> - Example: <br>
> Original → "I am a student, and I currently take MDS472C." <br>
> Preprocessed → ['i', 'am', 'a', 'student', 'and', 'i', 'currently', 'take', 'mds472c']

**3. Building the Vocabulary:** From all the tokenized documents, the program extracts a unique set of words (i.e., vocabulary). This is the list of terms to be checked in each document. <br>
> - Example Vocabulary: <br>
> ['a', 'am', 'and', 'can', 'cat', 'chase', 'currently', 'dog', ...]

**4. Constructing the Term-Document Matrix:** The program now creates a matrix where: <br>
> - Each row represents a word from the vocabulary. <br>
> - Each column shows the number of times that word appears in a document (its term frequency).

**5. Interactive Menu for the User:** The program gives a simple menu where the user can: <br>
> - View the full vocabulary list. <br>
> - Display the complete term-document matrix. <br>
> - Search for any word to see how many times it appears in each document. <br>
> - Exit the program.

# Question 3: 
**1. Collect the documents (atleast 2) to be indexed.** <br>
**2. Tokenize the text (convert each document into a list of tokens).** <br>
**3. Implement linguistic preprocessing (apply normalization, stemming, lemmatization, etc)** <br>
**4. Index the document with its frequency** <br>
**5. Sort the list (with frequency of words or sequence of words)** <br>
**6. Calculate the edit distance of any two words from the corpus.**

In [5]:
# Documents (You can replace or extend these)
documents = {
    "Doc1": "Natural language processing (NLP) is a fascinating area of artificial intelligence.",
    "Doc2": "Text preprocessing includes tokenization, stemming, lemmatization, and normalization."
}

# Preprocessing Functions
def normalize(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

def tokenize(text):
    return word_tokenize(text)

def stem_tokens(tokens):
    ps = PorterStemmer()
    return [ps.stem(token) for token in tokens]

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

# Step 1: Full preprocessing pipeline
def full_preprocess(docs):
    preprocessed = {}
    for doc_name, content in docs.items():
        print(f"\n{doc_name} Original: {content}")
        norm = normalize(content)
        tokens = tokenize(norm)
        stemmed = stem_tokens(tokens)
        lemmatized = lemmatize_tokens(tokens)
        preprocessed[doc_name] = {
            "normalized": norm,
            "tokens": tokens,
            "stemmed": stemmed,
            "lemmatized": lemmatized
        }
        print(f"Normalized: {norm}")
        print(f"Tokens: {tokens}")
        print(f"Stemmed: {stemmed}")
        print(f"Lemmatized: {lemmatized}")
    return preprocessed

# Step 2: Indexing word frequency
def index_frequency(all_tokens):
    flat_tokens = [token for tokens in all_tokens.values() for token in tokens['tokens']]
    freq = Counter(flat_tokens)
    return freq

# Step 3: Display frequency index
def display_freq(freq, sort_by="freq"):
    print("\nWORD FREQUENCY INDEX:")
    if sort_by == "freq":
        sorted_items = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    else:
        sorted_items = sorted(freq.items())
    for word, count in sorted_items:
        print(f"{word}: {count}")

# Step 4: Edit distance between any two words
def compute_edit_distance(w1, w2):
    dist = edit_distance(w1, w2)
    print(f"\nEdit Distance between '{w1}' and '{w2}': {dist}")

# Menu System
def main():
    print("🔧 Starting Preprocessing and Indexing...\n")
    processed = full_preprocess(documents)
    freq_index = index_frequency(processed)
    print("\nPreprocessing and Indexing Complete!")

    while True:
        print("\n======== MENU ========")
        print("1. Show All Tokenized + Stemmed + Lemmatized Outputs")
        print("2. Show Frequency Index (Sorted by Frequency)")
        print("3. Show Frequency Index (Sorted Alphabetically)")
        print("4. Calculate Edit Distance between Two Words")
        print("5. Exit")
        print("======================")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            for doc, items in processed.items():
                print(f"\n{doc}:")
                print(f"• Tokens: {items['tokens']}")
                print(f"• Stemmed: {items['stemmed']}")
                print(f"• Lemmatized: {items['lemmatized']}")
        elif choice == '2':
            display_freq(freq_index, sort_by="freq")
        elif choice == '3':
            display_freq(freq_index, sort_by="alpha")
        elif choice == '4':
            w1 = input("Enter first word: ").lower()
            w2 = input("Enter second word: ").lower()
            compute_edit_distance(w1, w2)
        elif choice == '5':
            print("Exiting.")
            break
        else:
            print("Invalid choice. Try again.")

# Run the program
main()

🔧 Starting Preprocessing and Indexing...


Doc1 Original: Natural language processing (NLP) is a fascinating area of artificial intelligence.
Normalized: natural language processing nlp is a fascinating area of artificial intelligence
Tokens: ['natural', 'language', 'processing', 'nlp', 'is', 'a', 'fascinating', 'area', 'of', 'artificial', 'intelligence']
Stemmed: ['natur', 'languag', 'process', 'nlp', 'is', 'a', 'fascin', 'area', 'of', 'artifici', 'intellig']
Lemmatized: ['natural', 'language', 'processing', 'nlp', 'is', 'a', 'fascinating', 'area', 'of', 'artificial', 'intelligence']

Doc2 Original: Text preprocessing includes tokenization, stemming, lemmatization, and normalization.
Normalized: text preprocessing includes tokenization stemming lemmatization and normalization
Tokens: ['text', 'preprocessing', 'includes', 'tokenization', 'stemming', 'lemmatization', 'and', 'normalization']
Stemmed: ['text', 'preprocess', 'includ', 'token', 'stem', 'lemmat', 'and', 'normal']
Lemmatized:

**Goal: Efficiently preprocess text from documents and analyze the words through stemming, lemmatization, and word frequency. Additionally, provide an option to compute the edit distance between any two words. This is useful for tasks in natural language processing like search, spell checking, or document comparison.**

**1. Documents to Process:** We start with two sample documents that contain sentences related to NLP and text preprocessing. <br>
> - Doc1: "Natural language processing (NLP) is a fascinating area of artificial intelligence." <br>
> - Doc2: "Text preprocessing includes tokenization, stemming, lemmatization, and normalization." <br>

**2. Preprocessing the Text: Each document goes through the following cleaning and processing steps:** <br>
> - Normalize: Convert to lowercase and remove punctuation. <br>
> - Tokenize: Split the cleaned sentence into individual words. <br>
> - Stem: Reduce each word to its root form (e.g., "running" → "run"). <br>
> - Lemmatize: Convert each word to its dictionary (base) form (e.g., "better" → "good"). <br>
> - This creates multiple versions of the text that are useful for different tasks.

**3. Index Word Frequencies:** From the tokenized version of the text, we count how many times each word appears across all documents.This frequency index is useful for identifying common vs. rare words in the dataset. <br>
> - Example Output: <br>
> WORD FREQUENCY INDEX: <br>
> of: 2  <br>
> processing: 1  <br>
> nlp: 1  <br>
> lemmatization: 1  <br>
> ...

**4. Compute Edit Distance:** The user can enter any two words (e.g., "token" and "taken"), and the program calculates the edit distance between them. This tells how many insertions, deletions, or substitutions are needed to turn one word into the other. <br>
> - Example: <br>
> Edit Distance between 'token' and 'taken': 2

**5. User Interaction via Menu:** The program gives a simple interactive menu where the user can: <br>
> - View tokens, stemmed forms, and lemmatized forms for all documents. <br>
> - View the frequency index (sorted by frequency or alphabetically). <br>
> - Compute edit distance between two words. <br>
> - Exit the program. <br>
> - This makes the tool flexible and interactive for quick text analysis tasks.

# Question 4:
**Consider the two words below:** <br>
	**Word A: characterization** <br>
	**Word B: categorization** <br><br>

**Using the Levenshtein Edit Distance algorithm, where each insertion, deletion, and substitution has a cost of 1, perform the following:** <br>
**1. Construct the Dynamic Programming (DP) matrix to compute the minimum edit distance between the two words.** <br>
**2. Trace the optimal path of operations used to transform Word A into Word B.** <br>
**3. Clearly specify each operation (Insert, Delete, Substitute, Match) along with the characters involved.** <br>
**4.Print the final aligned form of both words, indicating matched and edited characters.** <br>
**5. Report: Total Minimum Edit Distance, Number of Insertions, Number of Deletions, Number of Substitutions, Number of Matches** <br>


In [6]:
def compute_edit_distance(wordA, wordB):
    m, n = len(wordA), len(wordB)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    op = [[None] * (n + 1) for _ in range(m + 1)]

    # Initialize base cases
    for i in range(m + 1):
        dp[i][0] = i
        op[i][0] = 'D' if i > 0 else None
    for j in range(n + 1):
        dp[0][j] = j
        op[0][j] = 'I' if j > 0 else None

    # Fill DP table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if wordA[i - 1] == wordB[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
                op[i][j] = 'M'  # Match
            else:
                choices = [
                    (dp[i - 1][j - 1] + 1, 'S'),  # Substitute
                    (dp[i][j - 1] + 1, 'I'),      # Insert
                    (dp[i - 1][j] + 1, 'D')       # Delete
                ]
                dp[i][j], op[i][j] = min(choices)

    # Trace back
    i, j = m, n
    aligned_A = []
    aligned_B = []
    operations = []
    stats = {"I": 0, "D": 0, "S": 0, "M": 0}

    while i > 0 or j > 0:
        current_op = op[i][j]
        stats[current_op] += 1
        if current_op == 'M' or current_op == 'S':
            aligned_A.append(wordA[i - 1])
            aligned_B.append(wordB[j - 1])
            operations.append('*' if current_op == 'M' else 's')
            i -= 1
            j -= 1
        elif current_op == 'I':
            aligned_A.append('-')
            aligned_B.append(wordB[j - 1])
            operations.append('+')
            j -= 1
        elif current_op == 'D':
            aligned_A.append(wordA[i - 1])
            aligned_B.append('-')
            operations.append('-')
            i -= 1

    # Reverse to correct order
    aligned_A.reverse()
    aligned_B.reverse()
    operations.reverse()

    return dp, aligned_A, aligned_B, operations, stats, dp[m][n]

# === Run for Word A and Word B ===
wordA = "characterization"
wordB = "categorization"

dp_matrix, aligned_A, aligned_B, ops, stats, total_dist = compute_edit_distance(wordA, wordB)

# === Print Results ===
print("\nDynamic Programming (DP) Matrix:")
for row in dp_matrix:
    print(row)

print("\nAligned Words:")
print("Word A : ", ''.join(aligned_A))
print("Word B : ", ''.join(aligned_B))
print("Opertn : ", ''.join(ops))

print("\nOperation Stats:")
print(f"Matches      : {stats['M']}")
print(f"Substitutions: {stats['S']}")
print(f"Insertions   : {stats['I']}")
print(f"Deletions    : {stats['D']}")
print(f"\nTotal Minimum Edit Distance: {total_dist}")


Dynamic Programming (DP) Matrix:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[2, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[3, 2, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12]
[4, 3, 2, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 12]
[5, 4, 3, 3, 3, 4, 5, 6, 6, 7, 7, 8, 9, 10, 11]
[6, 5, 4, 4, 4, 4, 5, 6, 7, 7, 8, 8, 9, 10, 11]
[7, 6, 5, 4, 5, 5, 5, 6, 7, 8, 8, 8, 9, 10, 11]
[8, 7, 6, 5, 4, 5, 6, 6, 7, 8, 9, 9, 9, 10, 11]
[9, 8, 7, 6, 5, 5, 6, 6, 7, 8, 9, 10, 10, 10, 11]
[10, 9, 8, 7, 6, 6, 6, 7, 6, 7, 8, 9, 10, 11, 11]
[11, 10, 9, 8, 7, 7, 7, 7, 7, 6, 7, 8, 9, 10, 11]
[12, 11, 10, 9, 8, 8, 8, 8, 8, 7, 6, 7, 8, 9, 10]
[13, 12, 11, 10, 9, 9, 9, 9, 9, 8, 7, 6, 7, 8, 9]
[14, 13, 12, 11, 10, 10, 10, 10, 9, 9, 8, 7, 6, 7, 8]
[15, 14, 13, 12, 11, 11, 10, 11, 10, 10, 9, 8, 7, 6, 7]
[16, 15, 14, 13, 12, 12, 11, 11, 11, 11, 10, 9, 8, 7, 6]

Aligned Words:
Word A :  characterization
Word B :  c-atego-rization
Opertn :  *-*ssss-********

Operation 

**Goal: Efficiently compute the edit distance between two words using dynamic programming. This also includes alignment of characters, tracking operations (insertions, deletions, substitutions, matches), and visualizing the transformation from one word to another.**

**1. Words to Compare:** We work with two example words. These can be changed by the user to compare any two words. <br>
> - Example: <br>
> Word A: "characterization" <br>
> Word B: "categorization"

**2. Compute Edit Distance with Dynamic Programming:** A 2D table (dp) is built where dp[i][j] stores the minimum edit distance between the first i characters of Word A and first j characters of Word B. We also track the operation performed: <br>
> 'M' → Match (no cost) <br>
> 'S' → Substitute <br>
> 'I' → Insert <br>
> 'D' → Delete <br>

**3. Trace Back the Alignment:** After filling the dp table, we trace back from the bottom-right cell to reconstruct: <br>
> - The aligned version of both words. <br>
> - A sequence of operations showing how Word A becomes Word B. <br>
> - For example: <br>
> Word A :  chara-cterization <br>
> Word B :  catego-rization <br>
> Opertn :  *ss--**s********** <br>
> Here, * indicates match, s is substitution, - is deletion or insertion.

**4. Output the Statistics:** The program shows: <br>
> - How many matches, insertions, deletions, and substitutions were made. <br>
> - The total minimum edit distance between the two words. <br>


**5. Output the DP Matrix:**
> - To understand the process step-by-step, the program also prints the full DP matrix used to compute the edit distance. <br>
> - This is useful if you're analyzing the algorithm or debugging. <br>

### Question 5:  You are provided with a small training corpus. Your task is to identify suitable POS tags. Calculate transition and emission probabilities, and then tag a test sentence using HMM principles.** <br><br>
**Training Corpus:** <br>
**The cat chased the rat** <br>
**A rat can run** <br>
**The dog can chase the cat** <br>
**Test Case: “ The rat can chase the cat ”**


In [7]:
# Step 1: Training Data (with manual POS tagging)
training_sentences = [
    [("The", "DT"), ("cat", "NN"), ("chased", "VBD"), ("the", "DT"), ("rat", "NN")],
    [("A", "DT"), ("rat", "NN"), ("can", "MD"), ("run", "VB")],
    [("The", "DT"), ("dog", "NN"), ("can", "MD"), ("chase", "VB"), ("the", "DT"), ("cat", "NN")]
]

# Step 2: Count collection
transition_counts = defaultdict(Counter)
emission_counts = defaultdict(Counter)
tag_counts = Counter()

for sentence in training_sentences:
    prev_tag = "<START>"
    for word, tag in sentence:
        transition_counts[prev_tag][tag] += 1
        emission_counts[tag][word.lower()] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    transition_counts[prev_tag]["<STOP>"] += 1

# Step 3: Probability computation
def compute_probabilities(counts_dict):
    prob_dict = {}
    for key1 in counts_dict:
        total = sum(counts_dict[key1].values())
        prob_dict[key1] = {}
        for key2 in counts_dict[key1]:
            prob_dict[key1][key2] = counts_dict[key1][key2] / total
    return prob_dict

transition_probs = compute_probabilities(transition_counts)
emission_probs = compute_probabilities(emission_counts)

# Step 4: Print Transition Probabilities
print("\n--- Transition Probabilities ---")
for prev_tag in transition_probs:
    for curr_tag in transition_probs[prev_tag]:
        print(f"P({curr_tag} | {prev_tag}) = {transition_probs[prev_tag][curr_tag]:.4f}")

# Step 5: Print Emission Probabilities
print("\n--- Emission Probabilities ---")
for tag in emission_probs:
    for word in emission_probs[tag]:
        print(f"P({word} | {tag}) = {emission_probs[tag][word]:.4f}")

# Step 6: Viterbi Algorithm (Simple Probabilities)
test_sentence = ["the", "rat", "can", "chase", "the", "cat"]
tags = list(tag_counts.keys())

viterbi = [{}]
backpointer = [{}]

# Initialize for time step 0
for tag in tags:
    trans_p = transition_probs["<START>"].get(tag, 0.0001)
    emit_p = emission_probs[tag].get(test_sentence[0], 0.0001)
    viterbi[0][tag] = trans_p * emit_p
    backpointer[0][tag] = "<START>"

# Time step t > 0
for t in range(1, len(test_sentence)):
    viterbi.append({})
    backpointer.append({})
    for curr_tag in tags:
        max_prob = 0
        best_prev = None
        for prev_tag in tags:
            trans_p = transition_probs[prev_tag].get(curr_tag, 0.0001)
            emit_p = emission_probs[curr_tag].get(test_sentence[t], 0.0001)
            prob = viterbi[t-1][prev_tag] * trans_p * emit_p
            if prob > max_prob:
                max_prob = prob
                best_prev = prev_tag
        viterbi[t][curr_tag] = max_prob
        backpointer[t][curr_tag] = best_prev

# Final tag selection
n = len(test_sentence) - 1
(prob, final_tag) = max(
    ((viterbi[n][tag] * transition_probs[tag].get("<STOP>", 0.0001), tag) for tag in tags),
    key=lambda x: x[0]
)

# Backtracking
final_path = [final_tag]
for t in range(n, 0, -1):
    final_path.insert(0, backpointer[t][final_path[0]])

# Step 7: Final Output
print("\nFinal Tagged Sentence:")
for word, tag in zip(test_sentence, final_path):
    print(f"{word} → {tag}")

print(f"\nMost probable tag sequence: {final_path}")
print(f"Probability of the best path: {prob:.6f}")


--- Transition Probabilities ---
P(DT | <START>) = 1.0000
P(NN | DT) = 1.0000
P(VBD | NN) = 0.2000
P(<STOP> | NN) = 0.4000
P(MD | NN) = 0.4000
P(DT | VBD) = 1.0000
P(VB | MD) = 1.0000
P(<STOP> | VB) = 0.5000
P(DT | VB) = 0.5000

--- Emission Probabilities ---
P(the | DT) = 0.8000
P(a | DT) = 0.2000
P(cat | NN) = 0.4000
P(rat | NN) = 0.4000
P(dog | NN) = 0.2000
P(chased | VBD) = 1.0000
P(can | MD) = 1.0000
P(run | VB) = 0.5000
P(chase | VB) = 0.5000

Final Tagged Sentence:
the → DT
rat → NN
can → MD
chase → VB
the → DT
cat → NN

Most probable tag sequence: ['DT', 'NN', 'MD', 'VB', 'DT', 'NN']
Probability of the best path: 0.004096


**Goal: Build a Hidden Markov Model (HMM) for Part-of-Speech (POS) tagging, using:** <br>
- **Transition probabilities (between tags),**
- **Emission probabilities (word given a tag),**
- **And the Viterbi algorithm to determine the most likely tag sequence for a given sentence.**


**1. Words and Tags to Train From:** We start with manually tagged training sentences. These small examples define the tag sequences the model will learn from. <br>
> - Example training data: <br>
> [[("The", "DT"), ("cat", "NN"), ("chased", "VBD"), ("the", "DT"), ("rat", "NN")], [("A", "DT"), ("rat", "NN"), ("can", "MD"), ("run", "VB")],
  [("The", "DT"), ("dog", "NN"), ("can", "MD"), ("chase", "VB"), ("the", "DT"), ("cat", "NN")]]

**2. Count Collection:** We go through each training sentence and: <br>
> - Count transitions from one tag to the next (e.g., DT → NN, NN → VBD).
> - Count emissions of words given their tags (e.g., P("cat" | NN)).
> - Track tag frequency overall.
> - Special tags: <br>
> \<START> represents the beginning of a sentence. <br>
> \<STOP> is added after the last word in each sentence.

**3. Compute Probabilities:** From the counts: <br>
> - Transition Probability: <br>
> P(current_tag | previous_tag) = count(previous → current) / total transitions from previous
> - Emission Probability: <br>
> P(word | tag) = count(word emitted by tag) / total emissions of tag

**4. Output the Transition Probabilities:** We print the calculated values like:
> --- Transition Probabilities --- <br>
P(DT | <START>) = 1.0000   <br>
P(NN | DT) = 1.0000   <br>
P(VBD | NN) = 0.3333   <br>
...

**5. Output the Emission Probabilities:** Likewise, we show which words are most likely for each tag:
> --- Emission Probabilities --- <br>
P(the | DT) = 0.6000   <br>
P(a | DT) = 0.2000   <br>
P(cat | NN) = 0.3333   <br>
...

**6. Apply Viterbi Algorithm to Test Sentence:** We use the learned model to predict tags for a new sentence: <br>
> - Test Input: ["the", "rat", "can", "chase", "the", "cat"] <br>
> - Viterbi does the following: <br>
> Uses dynamic programming to find the most probable path (sequence of tags). <br>
> Keeps a backpointer to reconstruct the best path. <br>
> Handles unseen transitions/emissions with a small smoothing value (e.g., 0.0001). <br>

**7. Final Output: Tagged Sentence + Probability:** We display the predicted POS tags for the sentence and the total path probability.
> - Example output:
> Final Tagged Sentence:
the → DT  <br>
rat → NN  <br>
can → MD  <br>
chase → VB  <br>
the → DT   <br>
cat → NN   <br>

> Most probable tag sequence: ['DT', 'NN', 'MD', 'VB', 'DT', 'NN'] <br>
> Probability of the best path: 0.000032 <br>


# Question 6: Word Senses <br>
**(a.) Collect a small corpus of example sentences of varying lengths from any newspaper or magazine. Using WordNet or any standard dictionary, determine how many senses there are for each of the open-class words in each sentence.** <br>
**(b.) Implement Lesk algorithm for Word Sense Disambiguation (WSD)** <br>
**(c.) Using WordNet or a standard reference dictionary, tag each open-class word in your corpus with its correct tag.** <br>

In [8]:
# POS tag mapping from Penn Treebank to WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None

# Sample news/magazine sentences
sentences = [
    "The bank will not be open on Sunday.",
    "He went to the bank of the river to fish.",
    "The judge sentenced the criminal to five years in prison.",
    "The stock market crashed due to economic uncertainty."
]

# Stopwords to ignore
stop_words = set(stopwords.words('english'))

# Process each sentence
for idx, sentence in enumerate(sentences):
    print(f"\nSentence {idx + 1}: {sentence}")
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    
    print(f"POS Tags: {pos_tags}")
    print("Word Senses and Disambiguation:")

    for word, tag in pos_tags:
        wn_pos = get_wordnet_pos(tag)
        if wn_pos and word.lower() not in stop_words:
            senses = wn.synsets(word, pos=wn_pos)
            print(f"\n🔸 Word: {word}")
            print(f" - POS: {wn_pos}")
            print(f" - Total Senses: {len(senses)}")

            if senses:
                print(" - All Senses:")
                for i, s in enumerate(senses):
                    print(f"   [{i+1}] {s.name()} - {s.definition()}")

                # Lesk Disambiguation
                best_sense = lesk(tokens, word, pos=wn_pos)
                if best_sense:
                    print(f"Predicted Sense using Lesk: {best_sense.name()} - {best_sense.definition()}")
                else:
                    print("Lesk failed to disambiguate.")
            else:
                print("No senses found in WordNet.")


Sentence 1: The bank will not be open on Sunday.
POS Tags: [('The', 'DT'), ('bank', 'NN'), ('will', 'MD'), ('not', 'RB'), ('be', 'VB'), ('open', 'JJ'), ('on', 'IN'), ('Sunday', 'NNP'), ('.', '.')]
Word Senses and Disambiguation:

🔸 Word: bank
 - POS: n
 - Total Senses: 10
 - All Senses:
   [1] bank.n.01 - sloping land (especially the slope beside a body of water)
   [2] depository_financial_institution.n.01 - a financial institution that accepts deposits and channels the money into lending activities
   [3] bank.n.03 - a long ridge or pile
   [4] bank.n.04 - an arrangement of similar objects in a row or in tiers
   [5] bank.n.05 - a supply or stock held in reserve for future use (especially in emergencies)
   [6] bank.n.06 - the funds held by a gambling house or the dealer in some gambling games
   [7] bank.n.07 - a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
   [8] savings_bank.n.02 - a container (usu

**Goal: To analyze multiple meanings (word senses) of open-class words using WordNet, and disambiguate their correct meaning based on context using the Lesk Algorithm. This is useful in Word Sense Disambiguation (WSD), a key problem in Natural Language Processing.**

**1. Collect Sentences from a Magazine/News Article:** We use 4 example sentences: <br>
> Sentence 1: The bank will not be open on Sunday. <br>
> Sentence 2: He went to the bank of the river to fish. <br>
> Sentence 3: The judge sentenced the criminal to five years in prison. <br>
> Sentence 4: The stock market crashed due to economic uncertainty. <br>

**2. Preprocessing Steps:** Tokenize each sentence into words. <br>
> - Get POS tags for each word using nltk.pos_tag. <br>
> - Filter only open-class words: nouns, verbs, adjectives, adverbs. <br>
> - Map POS tags to WordNet format for accurate sense lookup. <br>

**3. Find Word Senses using WordNet:**
> - For each open-class word, get all possible meanings (called synsets) from WordNet. <br>
> - Display all senses with their definitions.

**4. Apply the Lesk Algorithm:**
> - The Lesk algorithm selects the most likely sense of a word based on overlapping words in the context. <br>
> - It returns the best-fitting synset (meaning).

**5. Output Format (Per Sentence):**
> - Original sentence <br>
> - Tokenized and POS-tagged words <br>
> - For each open-class word: <br>
> POS category <br>
> Total senses found in WordNet <br>
> List of all senses <br>
> Best sense predicted by Lesk algorithm <br>